# Extract differential expression analsys from Phase 2 Rdata

In [1]:
library(dplyr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




## Load data

In [2]:
lname = load('/ceph/users/jbenja13/phase3_paper/phase2/_m/deres.Rdata')
lname

[1] "deres"

In [3]:
names(deres)

[1] "development" "region"      "sczd"

## Extract features and select common columns

In [39]:
tx = deres$sczd$tx %>% as.data.frame %>% tibble::rownames_to_column("Feature") %>%
    mutate(Symbol=gene_name, gencodeID=gene_id, gencodeTx=transcript_id)
t_col = colnames(tx)

jxn = deres$sczd$jxn %>% tibble::rownames_to_column("Feature") %>%
    mutate(gencodeID=newGeneID, Symbol=newGeneSymbol)
j_col = colnames(jxn)

gene = deres$sczd$gene %>% as.data.frame %>% tibble::rownames_to_column("Feature")
g_col = colnames(gene)

exon = deres$sczd$exon %>% as.data.frame %>% tibble::rownames_to_column("Feature")
e_col = colnames(exon)

cols = intersect(intersect(intersect(j_col, e_col), g_col), t_col)
cols

[1] "Feature"      "ensemblID"    "Symbol"       "gencodeTx"    "meanExprs"   
 [6] "passExprsCut" "logFC"        "AveExpr"      "t"            "P.Value"     
[11] "adj.P.Val"    "B"            "region"       "type"         "gencodeID"

## Merge data

In [47]:
df = bind_rows(gene %>% select(cols, -passExprsCut, -gencodeTx),
               tx %>% select(cols, -passExprsCut, -gencodeTx), 
               exon %>% select(cols, -passExprsCut, -gencodeTx), 
               jxn %>% select(cols, -passExprsCut, -gencodeTx))
df %>% dim

[1] 1622296      13

## Write output

In [48]:
df %>% filter(region == "DLPFC") %>%
    data.table::fwrite("dlpfc_diffExpr_szVctl_full.tsv", sep='\t')

df %>% filter(region == "HIPPO") %>%
    data.table::fwrite("hippo_diffExpr_szVctl_full.tsv", sep='\t')

## Reproducibility Information

In [ ]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()